In [29]:
%pip install numpy
%pip install matplotlib
%pip install pandas


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import numpy as np
import pandas as pd
import random
import math

In [31]:
def create_individual(num_customers):
    customers = list(range(1, num_customers + 1))
    random.shuffle(customers)
    return customers

## Used ChatGPT
def split_customers_capacitated(customers, demands, capacity,  max_vehicles):
    routes = []
    current_route = []
    current_load = 0
    
    for cust_id in customers:
        demand = demands[cust_id - 1]
        if current_load + demand > capacity:
            routes.append(current_route)
            current_route = [cust_id]
            current_load = demand
        else:
            current_route.append(cust_id)
            current_load += demand

    
    if current_route:
        routes.append(current_route)

    
    # Simple repair: merge smallest routes until within limit
    while len(routes) > max_vehicles:
        # Pick two shortest routes and merge them
        r1 = min(routes, key=lambda r: sum(demands[c - 1] for c in r))
        routes.remove(r1)
        r2 = min(routes, key=lambda r: sum(demands[c - 1] for c in r))
        routes.remove(r2)

        merged = r1 + r2
        # If merged exceeds capacity, split again inside
        if sum(demands[c - 1] for c in merged) > capacity:
            # fallback: put back separately
            routes.append(r1)
            routes.append(r2)
            break
        else:
            routes.append(merged)

    return routes


def euclidean_distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])


def calculate_distance_matrix(depot, customer_coords):
    all_locations = [depot] + customer_coords
    N = len(all_locations)
    distance_matrix = [[0.0] * N for _ in range(N)]

    for i in range(N):
        for j in range(N):
            distance_matrix[i][j] = euclidean_distance(all_locations[i], all_locations[j])

    return distance_matrix


def evaluate_solution(routes, dist_matrix):
    route_lengths = []
    for route in routes:
        length = 0
        prev = 0 # depot index
        for cust_id in route:
            cust_index = cust_id
            length += dist_matrix[prev][cust_index]
            prev = cust_index
        length += dist_matrix[prev][0]
        route_lengths.append(length)
    
    total_distance = sum(route_lengths)
    max_route = max(route_lengths)
    avg = total_distance / len(route_lengths)
    std_dev = math.sqrt(sum((rl - avg)**2 for rl in route_lengths) / len(route_lengths))

    return np.array([total_distance, std_dev])


def non_dominated_sort(fitness_values: np.ndarray):
    pop_size = len(fitness_values)
    fronts = [[]]
    dominated_solutions = [[] for _ in range (pop_size)]
    domination_counts = [0] * pop_size

    for i in range(pop_size):
        for j in range(i+1, pop_size):
            if np.all(fitness_values[i] <= fitness_values[j]) and np.any(fitness_values[i] < fitness_values[j]):
                dominated_solutions[i].append(j)
                domination_counts[j] += 1
            elif np.all(fitness_values[j] <= fitness_values[i]) and np.any(fitness_values[j] < fitness_values[i]):
                dominated_solutions[j].append(i)
                domination_counts[i] += 1
        
        if domination_counts[i] == 0:
            fronts[0].append(i)
    
    k = 0
    while fronts[k]:
        next_front = []
        for i in fronts[k]:
            for j in dominated_solutions[i]:
                domination_counts[j] -= 1
                if domination_counts[j] == 0:
                    next_front.append(j)
        k += 1
        fronts.append(next_front)
    
    return fronts[:-1] 

def sharin_function(distance, niche_radius=1.0, alpha=1):

    if distance < niche_radius:
        return 1 - (distance / niche_radius) ** alpha
    return 0.0

def calculate_niche_count(fitness_values: np.ndarray, front: list, niche_radius=1.0) -> np.ndarray:
    n = len(front)
    niche_counts = np.zeros(n)

    for i in range(n):
        for j in range(n):
            if i != j:
                diff = fitness_values[front[i]] - fitness_values[front[j]]
                dist = np.sqrt(np.sum(diff**2))
                if dist < niche_radius:
                    niche_counts[i] += 1 - (dist / niche_radius)
    
    return np.array(niche_counts)

def order_crossover(parent1, parent2, pcrossover=0.7):
    size = len(parent1)

    if random.random() > pcrossover:
        return parent1.copy()
    
    child = [None] * size

    start, end = sorted(random.sample(range(size), 2))

    child[start:end] = parent1[start:end]

    p2_items = [customer for customer in parent2 if customer not in child]
    pos = 0
    for i in range(size):
        if child[i] is None:
            child[i] = p2_items[pos]
            pos += 1
    
    return child


In [32]:
data = pd.read_csv('A-n32-k5.csv')
coords = data[['x', 'y']]
depot = tuple(coords.iloc[0])
customer_coords = list(coords.iloc[1:].itertuples(index=False, name=None))
num_customers = len(customer_coords)
demands = list(data['demand'])

population_size = 30
population = [create_individual(num_customers) for _ in range(population_size)]

distances = calculate_distance_matrix(depot, customer_coords)

fitness_values = []
routes_population = []
for ind in population:
    routes = split_customers_capacitated(ind, demands, capacity=100, max_vehicles=5)
    fit = evaluate_solution(routes, distances)
    fitness_values.append(fit)
    routes_population.append(routes)

fitness_values = np.array(fitness_values)

fronts = non_dominated_sort(fitness_values)
pareto_front = fronts[0]

niche_counts = calculate_niche_count(fitness_values, pareto_front, niche_radius=100.0)

print("\nPareto front solutions (trade-off between total distance & balance):")
for i, idx in enumerate(pareto_front):
    print("solution", i+1, ":")
    print("Routes:", routes_population[idx])
    print("Fitness (TotalDist, STD dev):", fitness_values[idx])
    print("Route loads:", [sum(demands[c - 1] for c in r) for r in routes_population[idx]])
    print("Niche count:", niche_counts[i])


#print("Giant tour:", rand_list)
#print("Routes:", routes)
#print("Route loads:", [sum(demands[c - 1] for c in r) for r in routes])


Pareto front solutions (trade-off between total distance & balance):
solution 1 :
Routes: [[16, 28, 30, 25, 6, 4], [20, 14, 5, 8, 23, 3], [26, 21, 29, 2, 11, 12, 1], [31, 17, 7, 9, 27, 15, 10, 22], [18, 13, 24, 19]]
Fitness (TotalDist, STD dev): [2000.12914187   75.75896457]
Route loads: [81, 100, 88, 83, 49]
Niche count: 0.13374644699957994
solution 2 :
Routes: [[27, 12, 13, 21, 8, 16, 1, 14], [6, 24, 22, 23, 28, 4, 15, 30, 18, 17], [20, 11, 29, 9, 26], [25, 10, 5, 7, 3], [31, 2, 19]]
Fitness (TotalDist, STD dev): [1833.49581293  120.12606255]
Route loads: [99, 99, 77, 92, 34]
Niche count: 0.0
solution 3 :
Routes: [[7, 24, 17, 14, 2, 9, 12], [18, 25, 23, 6, 20, 16], [31, 27, 30, 26, 13, 5], [28, 21, 1, 22, 29, 8, 3], [10, 15, 19, 11, 4]]
Fitness (TotalDist, STD dev): [2078.93851182   39.80016332]
Route loads: [93, 100, 82, 92, 34]
Niche count: 0.13374644699957994
